In [1]:
import pickle
import boto3 
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date

In [2]:
with open("credentials.yaml", "r") as f:
    config = yaml.safe_load(f)

### 1. Cargar datos históricos de S3

In [3]:
def cargar_datos_s3(bucket, bucket_path):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key']
    )

    s3 = session.resource('s3')
    
    obj = s3.Object(bucket, bucket_path).get()['Body'].read()
    dataset = pickle.loads(obj)
    
    return dataset

In [4]:
session = boto3.Session(
    aws_access_key_id = config['s3']['aws_access_key_id'],
    aws_secret_access_key = config['s3']['aws_secret_access_key']
)

s3 = session.client('s3')

In [5]:
bucket = "aplicaciones-cd-1"
key = "ingesta/inicial/"

In [6]:
bucket_path = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents'][0]['Key']

In [7]:
bucket_path

'ingesta/inicial/inspecciones-historicas-2020-11-16.pkl'

In [8]:
dataset = cargar_datos_s3(bucket, bucket_path)

In [9]:
len(dataset)

212910

### 2. Transformar datos a DF

In [10]:
def transformar_ingesta(dataset):
    return pd.DataFrame.from_dict(dataset)

In [11]:
df_inspecciones = transformar_ingesta(dataset)

In [12]:
df_inspecciones.shape

(212910, 17)

In [13]:
df_inspecciones.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '-87.67596676683779', 'longitude'...",NaN
1,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '-87.67596676683779', 'longitude'...",NaN
2,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631,2010-01-04T00:00:00.000,License,Fail,NaN,NaN,NaN,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
3,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04T00:00:00.000,Tag Removal,Pass,41.88458626715456,-87.63101044588599,"{'latitude': '-87.63101044588599', 'longitude'...",NaN
4,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04T00:00:00.000,Canvass,Fail,41.89843137207629,-87.6280091630558,"{'latitude': '-87.6280091630558', 'longitude':...",18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...


### 3. Identificar faltantes

In [14]:
def faltantes(df):
    return df.isna().sum()

In [15]:
faltantes(df_inspecciones)

inspection_id          0
dba_name               0
aka_name            2484
license_              17
facility_type       4895
risk                  72
address                0
city                 155
state                 48
zip                   53
inspection_date        0
inspection_type        1
results                0
latitude             703
longitude            703
location             703
violations         56674
dtype: int64

### 4. Eliminar inspecciones que no  tienen latitud o longitud

In [16]:
def elimina_faltantes_latitud_logintud(cols, df):
    for col in cols:
        df =  df[df[col].notnull()]
    
    return df

In [17]:
inspecciones = elimina_faltantes_latitud_logintud(['latitude', 'longitude'], df_inspecciones)

In [18]:
faltantes(inspecciones)

inspection_id          0
dba_name               0
aka_name            2472
license_              17
facility_type       4880
risk                  70
address                0
city                 151
state                 48
zip                   49
inspection_date        0
inspection_type        1
results                0
latitude               0
longitude              0
location               0
violations         56485
dtype: int64

### 5. Imputar faltantes

In [19]:
def imputar_faltantes(col, value, df):
    df[col].fillna(value, inplace=True)

In [40]:
# license_
imputar_faltantes('license_', str(inspecciones.license_.mode()), inspecciones)
# zip
imputar_faltantes('zip', str(inspecciones.zip.mode()), inspecciones)
# state
imputar_faltantes('state', str(inspecciones.state.mode()), inspecciones)
# facility_type
imputar_faltantes('facility_type', str(inspecciones.facility_type.mode()), inspecciones)
# risk
imputar_faltantes('risk', str(inspecciones.risk.mode()), inspecciones)

In [41]:
faltantes(inspecciones)

inspection_id          0
dba_name               0
aka_name            2472
license_               0
facility_type          0
risk                   0
address                0
city                 151
state                  0
zip                    0
inspection_date        0
inspection_type        1
results                0
latitude               0
longitude              0
location               0
violations         56485
dtype: int64

### 6. Transformación de enteros

In [42]:
def transformar_enteros(cols, df):
    for element in cols:
        df[element] = df[element].astype(int)
    
    return df

In [43]:
inspecciones = transformar_enteros(['inspection_id'], inspecciones)

In [44]:
inspecciones.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                  float64
longitude                 float64
location                   object
violations                 object
dtype: object

### 7. Transformación flotantes

In [45]:
def transformar_flotantes(cols, df):
    for col in cols:
        df[col] = df[col].astype(float)
    
    return df

In [46]:
inspecciones = transformar_flotantes(['latitude', 'longitude'], inspecciones)

In [47]:
inspecciones.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                  float64
longitude                 float64
location                   object
violations                 object
dtype: object

### 8. Transformaciones fechas

In [48]:
def transformar_fechas(cols, df):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
    
    return df

In [49]:
inspecciones = transformar_fechas(['inspection_date'], inspecciones)

In [50]:
inspecciones.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                  float64
longitude                 float64
location                   object
violations                 object
dtype: object

### 9. Data profiling categorías

In [51]:
def data_profiling_string(cols, df):
    data_profiling = {}
    for col in cols:
        data_profiling[col] = {'uniques': df[col].nunique(), 
                               'prop_uniques': df[col].nunique()/df[col].shape[0],
                              'mode': df[col].mode()} 

    return pd.DataFrame.from_dict(data_profiling)

In [52]:
data_profiling_string(['dba_name', 'aka_name', 'license_'], inspecciones)

,dba_name,aka_name,license_
uniques,28474,27099,38739
prop_uniques,0.13418,0.127701,0.182553
mode,0 SUBWAY dtype: object,0 SUBWAY dtype: object,0 0 dtype: object


### 10. Data profiling fechas

In [53]:
def data_profiling_fechas(cols, df):
    data_profiling = {}
    for col in cols:
        data_profiling[col] = {'uniques': df[col].nunique(), 
                               'prop_uniques': df[col].nunique()/df[col].shape[0],
                              'fecha_minima': df[col].min(),
                              'fecha_maxima': df[col].max(),
                              'anios_diferentes': df[col].dt.year.nunique(),
                              'dias_diferentes': df[col].max() - df[col].min()} 

    return pd.DataFrame.from_dict(data_profiling)

In [54]:
data_profiling_fechas(['inspection_date'], inspecciones)

,inspection_date
anios_diferentes,11
dias_diferentes,3966 days 00:00:00
fecha_maxima,2020-11-13 00:00:00
fecha_minima,2010-01-04 00:00:00
prop_uniques,0.0129779
uniques,2754


### 11. Guardar datos limpios en s3

In [55]:
def guardar_datos_s3(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key']
    )

    s3 = session.resource('s3')

    s3.Object(bucket, bucket_path).put(Body=dataset)

In [56]:
TODAY = date.today()

In [57]:
pickle_data = pickle.dumps(inspecciones)

In [58]:
bucket = "aplicaciones-cd-1"
key = "limpieza/datos-limpios-" + str(TODAY) + ".pkl"

guardar_datos_s3(bucket, key, pickle_data)